In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(
      name="ml-playground-01",
      subscription_id="1a342961-1209-4b7b-9e44-318aeea934e8",
      resource_group="DataEngineering",
)

print(ws)

exp = Experiment(workspace=ws, name="udacity-project-01")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_target = ws.compute_targets["testing-123"]

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, loguniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "--C": uniform(0.01, 1),
        "--max_iter": choice([10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000]),
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
from azureml.core import Environment

sklearn_env = Environment.get(workspace=ws, name='AzureML-Tutorial')

from azureml.core import ScriptRunConfig

path_to_script = "." # "~/cloudfiles/code/Users/ballen/udacity/nd00333_AZMLND_Optimizing_a_Pipeline_in_Azure-Starter_Files"
src = ScriptRunConfig(source_directory=path_to_script,
                      script='train.py',
                      arguments=[],
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hd_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="AUC",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=144,
    max_concurrent_runs=12,
)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hd_config)


In [ ]:
RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
# run.complete()

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print(parameter_values)

print(best_run.get_file_names())

# model = best_run.register_model(model_name='best_logistic', model_path='outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path)

In [ ]:
import train

# Use the clean_data function to clean your data.
x, y = train.clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_hours=6,
    compute_target=compute_target,
    task="classification",
    primary_metric="AUC_weighted",
    training_data=ds,
    label_column_name="y",
    n_cross_validations=10,
    enable_stack_ensemble=False,
    enable_voting_ensemble=False,
)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

In [ ]:
automl_run.wait_for_completion(show_output=True)

In [ ]:
automl_run.complete()

In [ ]:
run.complete()

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = automl_run.get_output()
print(fitted_model.steps)

model_name = best_run.properties['model_name']
description = 'AutoML forecast example'
tags = None

model = best_run.register_model(model_name = model_name, 
                                  description = description, 
                                  tags = tags)